In [1]:
import numpy as np
import math
from scipy import integrate
import matplotlib.pyplot as plt
import random
import time

In [2]:
def chi2(p,q,r,s,t,u):
    stat = 0
    if p+q != 0:
        stat += (p-q)**2/(p+q)
    if r+s != 0:
        stat += (r-s)**2/(r+s)
    if t+u != 0:
        stat += (t-u)**2/(t+u)
    return stat

In [3]:
def LS(p,q,r,s,t,u): #local sensitivity
    stat = chi2(p,q,r,s,t,u); v = np.zeros(0)
    M = chi2(p,q,r,s,t,u); m = chi2(p,q,r,s,t,u)
    if p >= 1:
        v = [chi2(p-1,q+1,r,s,t,u), chi2(p-1,q,r+1,s,t,u), chi2(p-1,q,r,s+1,t,u), 
             chi2(p-1,q,r,s,t+1,u), chi2(p-1,q,r,s,t,u+1)]
        M = max([max(v),M])
        m = min([min(v),m])
    if q >= 1:
        v = [chi2(p+1,q-1,r,s,t,u), chi2(p,q-1,r+1,s,t,u), chi2(p,q-1,r,s+1,t,u), 
             chi2(p,q-1,r,s,t+1,u), chi2(p,q-1,r,s,t,u+1)]
        M = max([max(v),M])
        m = min([min(v),m])
    if r >= 1:
        v = [chi2(p+1,q,r-1,s,t,u), chi2(p,q+1,r-1,s,t,u), chi2(p,q,r-1,s+1,t,u), 
             chi2(p,q,r-1,s,t+1,u), chi2(p,q,r-1,s,t,u+1)]
        M = max([max(v),M])
        m = min([min(v),m])
    if s >= 1:
        v = [chi2(p+1,q,r,s-1,t,u), chi2(p,q+1,r,s-1,t,u), chi2(p,q,r+1,s-1,t,u), 
             chi2(p,q,r,s-1,t+1,u), chi2(p,q,r,s-1,t,u+1)]
        M = max([max(v),M])
        m = min([min(v),m])
    if t >= 1:
        v = [chi2(p+1,q,r,s,t-1,u), chi2(p,q+1,r,s,t-1,u), chi2(p,q,r+1,s,t-1,u), 
             chi2(p,q,r,s+1,t-1,u), chi2(p,q,r,s,t-1,u+1)]
        M = max([max(v),M])
        m = min([min(v),m])
    if u >= 1:
        v = [chi2(p+1,q,r,s,t,u-1), chi2(p,q+1,r,s,t,u-1), chi2(p,q,r+1,s,t,u-1), 
             chi2(p,q,r,s+1,t,u-1), chi2(p,q,r,s,t+1,u-1)]
        M = max([max(v),M])
        m = min([min(v),m])
    
    return max([M-stat, stat-m])

In [4]:
def SS(table,beta): #smooth sensitivity
    p = table[0]; q = table[1]; r = table[2]; s = table[3]; t = table[4]; u = table[5]
    GS = 4; ls = LS(p,q,r,s,t,u)
    if beta >= math.log(3):
        return ls
    else:
        dist = int(math.ceil(math.log(GS/ls))/beta)
        ss = 0
        for yp in range(-dist,dist+1):
            for yq in range(-dist,dist+1):
                for yr in range(-dist,dist+1):
                    for ys in range(-dist,dist+1):
                        for yt in range(-dist,dist+1):
                            yu = -yp-yq-yr-ys-yt
                            if p+yp >= 0 and q+yq >= 0 and r+yr >= 0 and s+ys >= 0 and t+yt >= 0 and u+yu >= 0:
                                dxy = (math.fabs(yp) + math.fabs(yq) + math.fabs(yr) + math.fabs(ys) + math.fabs(yt) + math.fabs(yu))/2
                                if dxy <= dist:
                                    ss = max([LS(p+yp,q+yq,r+yr,s+ys,t+yt,u+yu)*math.exp(-(beta*dxy)), ss])
        return ss

In [5]:
def generateData(n):
    table = np.zeros(6)
    table[0] = np.random.binomial(n,1/6)
    table[1] = np.random.binomial(n-table[0],1/5)
    table[2] = np.random.binomial(n-table[0]-table[1],1/4)
    table[3] = np.random.binomial(n-table[0]-table[1]-table[2],1/3)
    table[4] = np.random.binomial(n-table[0]-table[1]-table[2]-table[3],1/2)
    table[5] = n-table[0]-table[1]-table[2]-table[3]-table[4]
    return table

In [6]:
def RunTime(N,beta):
    t = 0
    for i in range(100):
        table = generateData(N)
        s = time.time()
        SS(table,beta)
        e = time.time()
        t += (e-s)
    return t/100

In [7]:
N = 1000; epsilon = [2,4,6,8]
for i in range(4):
    print(RunTime(N,epsilon[i]/4))

2.8925236773490908
0.129257972240448
5.508661270141602e-05
5.206108093261719e-05


In [8]:
N = 2000; epsilon = [2,4,6,8]
for i in range(4):
    print(RunTime(N,epsilon[i]/4))

4.800521402359009
0.24805015563964844
5.1178932189941405e-05
5.0580501556396486e-05


In [9]:
N = 5000; epsilon = [2,4,6,8]
for i in range(4):
    print(RunTime(N,epsilon[i]/4))

8.201853659152984
0.3398132300376892
5.126237869262695e-05
5.5985450744628905e-05
